# 0. Install Dependencies





In [27]:
#pip install rl-agents==0.1.1
#!pip install tensorflow
#!pip install gym
#!pip install keras
#!pip install keras-rl2
#!pip install pygame

# 1. Test Random Environment with OpenAI Gym




In [28]:
import gym
import random

In [29]:
#Game being used
env = gym.make('CartPole-v1')# -> gym.openai.com/#envs

states = env.observation_space.shape[0]

#Number of actions available. In the current case there are only 2 (left or right) that will move the cart
actions = env.action_space.n

In [30]:
print(states)
print(actions)

4
2


In [31]:
#Env setting with random behaviour
episodes = 10
for episode in range(1,episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1])
        #print(env.step(action))
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode,score))

env.close()

Episode:1 Score:30.0
Episode:2 Score:23.0
Episode:3 Score:35.0
Episode:4 Score:15.0
Episode:5 Score:25.0
Episode:6 Score:30.0
Episode:7 Score:24.0
Episode:8 Score:26.0
Episode:9 Score:12.0
Episode:10 Score:14.0


# 2. Create a Deep Learning Model with Keras

In [32]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.optimizers.legacy import Adam

from rl.agents import DQNAgent #check diferent agents -> https://keras-rl.readthedocs.io/en/latest/
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [33]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,4)))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(actions,activation='linear'))
    return model

In [34]:
model = build_model(states,actions)

In [35]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 4)                 0         
                                                                 
 dense_15 (Dense)            (None, 24)                120       
                                                                 
 dense_16 (Dense)            (None, 24)                600       
                                                                 
 dense_17 (Dense)            (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [36]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000,window_length=1)
    dqn = DQNAgent(model=model,memory=memory,policy=policy,
                  nb_actions=actions,nb_steps_warmup=10,target_model_update=1e-2)
    return dqn

In [37]:
dqn = build_agent(model,actions)
#Adam._name = 'Adam'

dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
#I had to train the model with twice the number of steps(comparing to the tutorial) to achieve the expected result
dqn.fit(env,nb_steps=100000,visualize=False,verbose=1) 

Training for 100000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 19:38 - reward: 1.0000

C:\Users\jmimoso\AppData\Local\anaconda3\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 83s 8ms/step - reward: 1.0000
106 episodes - episode_reward: 94.198 [10.000, 288.000] - loss: 1.681 - mae: 19.242 - mean_q: 39.080

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 91s 9ms/step - reward: 1.0000
51 episodes - episode_reward: 193.412 [151.000, 285.000] - loss: 2.638 - mae: 38.050 - mean_q: 77.010

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 70s 7ms/step - reward: 1.0000
44 episodes - episode_reward: 227.477 [165.000, 390.000] - loss: 1.909 - mae: 40.231 - mean_q: 81.237

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 70s 7ms/step - reward: 1.0000
37 episodes - episode_reward: 264.135 [204.000, 427.000] - loss: 1.820 - mae: 45.355 - mean_q: 91.661

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 67s 7ms/step - reward: 1.0000
24 episodes - episode_reward: 429.875 [226.000, 500.000] - loss:

In [38]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500
Episode 11: reward: 500.000, steps: 500
Episode 12: reward: 500.000, steps: 500
Episode 13: reward: 500.000, steps: 500
Episode 14: reward: 500.000, steps: 500
Episode 15: reward: 500.000, steps: 500
Episode 16: reward: 500.000, steps: 500
Episode 17: reward: 500.000, steps: 500
Episode 18: reward: 500.000, steps: 500
Episode 19: reward: 500.000, steps: 500
Episode 20: reward: 500.000, steps: 500
Episode 21: reward: 500.000, steps: 500
Episode 22: reward: 500.000, steps: 500
Episode 23: reward: 500.000, steps: 500
Episode 24: reward: 500.000, steps: 500
Episode 25: reward: 

In [39]:
#lets test again with more 15 episodes
_ = dqn.test(env, nb_episodes=15, visualize=True)


Testing for 15 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500
Episode 11: reward: 500.000, steps: 500
Episode 12: reward: 500.000, steps: 500
Episode 13: reward: 500.000, steps: 500
Episode 14: reward: 500.000, steps: 500
Episode 15: reward: 500.000, steps: 500


# 4. Reload Agent from Memory

In [40]:
dqn.save_weights('dqn_carpolev0_weights.h5f',overwrite=True)

In [41]:
del model
del dqn 
del env

In [42]:
env = gym.make('CartPole-v1') 

actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)

dqn = build_agent(model,actions)
dqn.compile(Adam(learning_rate=1e-3),metrics=['mae'])

Exception ignored in: <function Viewer.__del__ at 0x00000263EDF4A7A0>
Traceback (most recent call last):
  File "C:\Users\jmimoso\AppData\Local\anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 165, in __del__
    self.close()
  File "C:\Users\jmimoso\AppData\Local\anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 83, in close
    self.window.close()
  File "C:\Users\jmimoso\AppData\Local\anaconda3\lib\site-packages\pyglet\window\win32\__init__.py", line 299, in close
    super(Win32Window, self).close()
  File "C:\Users\jmimoso\AppData\Local\anaconda3\lib\site-packages\pyglet\window\__init__.py", line 823, in close
    app.windows.remove(self)
  File "C:\Users\jmimoso\AppData\Local\anaconda3\lib\_weakrefset.py", line 114, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x00000263F465FEC0; to 'Win32Window' at 0x00000263F493E7D0>


In [43]:
dqn.load_weights('dqn_carpolev0_weights.h5f')

In [44]:
#lets test again with more 15 episodes
_ = dqn.test(env, nb_episodes=15, visualize=True)
env.close()

Testing for 15 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500
Episode 11: reward: 500.000, steps: 500
Episode 12: reward: 500.000, steps: 500
Episode 13: reward: 500.000, steps: 500
Episode 14: reward: 500.000, steps: 500
Episode 15: reward: 500.000, steps: 500
